In [1]:
import numpy as np 
import pandas as pd
import os
import json


In [2]:
locationfile = pd.read_csv('location.csv')
accountfile = pd.read_csv('account.csv')

## Original Location & Account Files with fields 'FlexiLocUnit' & 'FlexiAccUnit' 


In [3]:
#Orignal Location File
locationfile.head(5)

,FlexiLocUnit,FlexiLoc_TestCase,PortNumber,AccNumber,LocNumber,CountryCode,Latitude,Longitude,PostalCode,GeogScheme1,...,CorrelationGroup,YearBuilt,NumberOfStoreys,Basement,FlexiLocSBUValve,FlexiLoc_ExpectedGrossLossDR100,LocDedType2Other,LocDed2Other,LocLimitType2Other,LocLimit2Other
0,fm3,fm3,fm3,1,1,CA,0,0,0,0,...,0,0,0,0,0,0,0,5000,0,90000
1,fm12,fm12,fm12,105449,23039308,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,fm12,fm12,fm12,105449,23039309,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,fm12,fm12,fm12,105449,23039311,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,fm12,fm12,fm12,105449,23039320,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#Orignal Account
accountfile.head(5)

,FlexiAccUnit,FlexiAcc_TestCase,PortNumber,AccNumber,PolNumber,PolPerilsCovered,AccCurrency,LayerParticipation,LayerLimit,LayerAttachment,...,CondNumber,CondDed6All,CondDedCode6All,CondDedType6All,CondMinDed6All,CondMaxDed6All,CondLimit6All,CondLimitCode6All,CondLimitType6All,CondPriority
0,fm3,fm3,fm3,1,1,OO1,EUR,1.00,0,0,...,0,0,0,0,0,0,0,0,0,0
1,fm12,fm12,fm12,105449,477353,OO1,EUR,0.15,5700000,0,...,1160,0,0,0,10000,0,100000000,0,0,1
2,fm12,fm12,fm12,105449,477353,OO1,EUR,0.15,5700000,0,...,1161,0,0,0,600000,0,15000000,0,0,1
3,fm12,fm12,fm12,105449,477353,OO1,EUR,0.15,5700000,0,...,1163,0,0,0,600000,0,15000000,0,0,1
4,fm12,fm12,fm12,105449,477354,OO1,EUR,0.15,4300000,5700000,...,1160,0,0,0,10000,0,100000000,0,0,1


## Split the records by the unique values in fields FlexiLocUnit & FlexiAccUnit


In [5]:
split_location = locationfile.sort_values('FlexiLocUnit').groupby('FlexiLocUnit')
split_account = accountfile.sort_values('FlexiAccUnit').groupby('FlexiAccUnit')

In [6]:
split_location

### loop through the groups and save the split dataframes to sub-directories based on their unique values

### Maintain parent-file name. e.g units\fm12\location.csv


In [7]:
newpath = 'units' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [8]:
cwd = os.getcwd()
for name, group in split_location:
    sub_dir = os.path.join(newpath,name)
    print (sub_dir)

units/fm12
units/fm3


In [18]:
cwd = os.getcwd()

#loop through the groups and save to directories based on unique values
for name, group in split_location:
    sub_dir = os.path.join(newpath,name)
    if not os.path.exists(sub_dir):
        os.mkdir(sub_dir)
    group = group.drop(['FlexiLocUnit'], axis=1)
    group.to_csv(sub_dir + "/location.csv", index=0)

for name, group in split_account:
    sub_dir = os.path.join(newpath,name)
    if not os.path.exists(sub_dir):
        os.mkdir(sub_dir)
    group = group.drop(['FlexiAccUnit'], axis=1)
    group.to_csv(sub_dir + "/account.csv", index=0)


In [9]:
names = sorted([str(item[0]) for item in split_location])



In [11]:
#Function to sort fm string in Ascedning order 
import re

def ascedning(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):

    return [ ascedning(c) for c in re.split(r'(\d+)', text) ]


names.sort(key=natural_keys)
#print(names)

In [12]:
# create units text file

In [13]:
units_dir=os.path.join(cwd,'units')

if not os.path.exists(units_dir):
    os.mkdir(units_dir)
    
with open(os.path.join(units_dir,'units.txt'), "w") as txt_file:
    names, groups = map(list, zip(*split_location))
    names.sort(key=natural_keys)
    for name in names:
        txt_file.write(str(name) + '\n')


In [14]:
# combine files back together

In [16]:
# get directories
with open(os.path.join(units_dir,'units.txt'), "r") as txt_file:
    fms = txt_file.read().split('\n')
    
dirs = []
for fm in fms:
    if fm!='':
        dirs.append(fm)

dirs


['fm3', 'fm12']

## Combine Units

In [17]:
# combine dataframes

# start with first one
fm_first = dirs[0]
fm_first_filepath = os.path.join(newpath,fm_first,'location.csv')

df_loc = pd.read_csv(fm_first_filepath)
df_loc['FlexiLocUnit']=fm_first

# add in remaining fm files, iterating through remainder
for i in range(1,len(dirs)):
    fm_next = dirs[i]
    fm_next_filepath = os.path.join(newpath,fm_next,'location.csv')
    df_loc_tmp = pd.read_csv(fm_next_filepath)
    df_loc_tmp['FlexiLocUnit']=fm_next
    # concat files
    df_loc = pd.concat([df_loc,df_loc_tmp])
    
df_loc.to_csv('location_concat.csv',index=False)


In [21]:
#Account concat

fm2_first = dirs[0]
fm2_first_filepath = os.path.join(newpath,fm2_first,'account.csv')

df_loc2 = pd.read_csv(fm2_first_filepath)
df_loc2['FlexiAccUnit']=fm2_first

# add in remaining fm files, iterating through remainder
for i in range(1,len(dirs)):
    fm2_next = dirs[i]
    fm2_next_filepath = os.path.join(newpath,fm2_next,'account.csv')
    df_loc_tmp2 = pd.read_csv(fm2_next_filepath)
    df_loc_tmp2['FlexiAccUnit']=fm2_next
    # concat files
    df_loc2 = pd.concat([df_loc2,df_loc_tmp2])
    
df_loc2.to_csv('account_concat.csv',index=False)